In [2]:
import requests
import os
from tqdm.auto import tqdm

C:\Program Files\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Пример скачивания 1 файла

In [4]:
# Full path - пример
# https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet

os.makedirs("nyc_taxi_data", exist_ok=True) # Создаем папку

base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data" # базовый путь к файлу на сервере
filename = f"yellow_tripdata_2024-01.parquet"                # имя файла
url = f"{base_url}/{filename}"                               # Полный URL
filepath = os.path.join("nyc_taxi_data", filename)  # Путь на диске (корректное объединение пути с помощью os.path.join())

In [5]:
# весь блок одеваем в try except, 
# чтобы при случае отсутствия соединения или файла программа не крашнулась
try:
    response = requests.get(url, stream=True)  # 1. Отправляем HTTP GET запрос к URL для скачивания файла
                                        #     stream=True - КРИТИЧЕСКИ важный параметр:
                                        # Без stream=True: requests загружает ВЕСЬ файл в оперативную память сразу
                                        #   С stream=True: файл загружается ЧАСТЯМИ (чанками)
    
    response.raise_for_status() # 2. ПРОВЕРКА НА ОШИБКИ HTTP: автоматически проверяет код ответа сервера
                                # - Если код 200 (OK) - ничего не происходит, продолжаем выполнение
                                # - Если код 4xx (клиентская ошибка) или 5xx (серверная ошибка) - 
                                #   ВЫБРАСЫВАЕТСЯ ИСКЛЮЧЕНИЕ requests.exceptions.HTTPError
                                
    with open(filepath, 'wb') as f: #  ОТКРЫВАЕМ ФАЙЛ ДЛЯ ЗАПИСИ в БИНАРНОМ РЕЖИМЕ:
                                    #  'wb' означает:
                                    #    w - write (запись)
                                    #    b - binary (бинарный режим - для файлов, которые не являются текстом)
                                    #  Конструкция 'with' гарантирует, что файл будет КОРРЕКТНО ЗАКРЫТ 
                                    #  даже если возникнет ошибка во время записи
        for chunk in response.iter_content(chunk_size=8192):    # ПОСТРОЧНОЕ ЧТЕНИЕ И ЗАПИСА ФАЙЛА:
                                                                # response.iter_content() - это ГЕНЕРАТОР, который возвращает файл 
                                                                # небольшими частями (чанками) вместо одной большой порции
                                                                # chunk_size=8192 - размер одного кусочка в байтах (8 КБ)
            if chunk:  # 5. ФИЛЬТРАЦИЯ ПУСТЫХ ЧАНКОВ:
                       # Иногда iter_content может возвращать пустые чанки (None или b'')
                       # Эта проверка гарантирует, что мы записываем ТОЛЬКО данные
                       # и не выполняем лишних операций записи
                f.write(chunk)  # 6. ЗАПИСЬ ЧАНКА НА ДИСК:
                                # Каждый кусочек данных (8 КБ) записывается в файл
                                # Это происходит МНОГОКРАТНО, пока весь файл не будет скачан
    
    file_size = os.path.getsize(filepath) / (1024 * 1024) # os.path.getsize(filepath) - получает размер файла в БАЙТАХ
    result = f"✓ {filename} ({file_size:.1f} MB)"
    
except Exception as e:
    result = f"✗ {filename}: {e}"

print(f'Результат: {result}')

Результат: ✓ yellow_tripdata_2024-01.parquet (47.6 MB)


## Скачиваем пакетом

In [7]:
def download_simple():
    """Упрощенная загрузка файлов по очереди"""
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data"
    os.makedirs("nyc_taxi_data", exist_ok=True)
    
    results = []
    
    for month in tqdm(range(1, 13), desc="Скачивание файла"):
        filename = f"yellow_tripdata_2024-{month:02d}.parquet"
        url = f"{base_url}/{filename}"
        filepath = os.path.join("nyc_taxi_data", filename)
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            file_size = os.path.getsize(filepath) / (1024 * 1024)
            results.append(f"✓ {filename} ({file_size:.1f} MB)")
            
        except Exception as e:
            results.append(f"✗ {filename}: {e}")
    
    return results

print("🚕 Загрузка NYC Yellow Taxi 2024 (упрощенная версия)...")
results = download_simple()

print("\nРезультаты:")
for result in results:
    print(result)

🚕 Загрузка NYC Yellow Taxi 2024 (упрощенная версия)...


Скачивание файла: 100%|██████████| 3/3 [00:03<00:00,  1.12s/it]


Результаты:
✓ yellow_tripdata_2024-12.parquet (58.7 MB)
✗ yellow_tripdata_2024-13.parquet: 403 Client Error: Forbidden for url: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-13.parquet
✗ yellow_tripdata_2024-14.parquet: 403 Client Error: Forbidden for url: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-14.parquet
